# Running an ABOR using Luminesce

In this notebook, we setup LUSID to run a Trial Balance on an Equity/Bond portfolio.

## Preliminary setup

Before we start the portfolio and ABOR setup, we need to create a Recipe and some Transaction Types in LUSID.

In [ ]:
# Import general modules
import logging
import lumipy as lm
import lusid
import lusid.models as models
import json
import os
import random
import pandas as pd
import numpy as np

# Import LUSID Drive modules
from lusid.utilities import ApiClientFactory as LusidApiClientFactory
from lusidjam import RefreshingToken

# Create loggers
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()


def create_recipe(api_factory):
    recipes_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

    scope = "luminesce-examples"
    recipe_code = "marketValue"

    configuration_recipe = models.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="100D.0D",
                ),
                models.MarketDataKeyRule(
                    key="FX.*.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="100D.0D",
                ),
            ],
            suppliers=models.MarketContextSuppliers(
                commodity="Client",
                credit="Client",
                equity="Client",
                fx="Client",
                rates="Client",
            ),
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="ClientInternal",
                default_scope=scope,
                attempt_to_infer_missing_fx=True,
            ),
        ),
        pricing=models.PricingContext(
            model_rules=[
                models.VendorModelRule(
                    supplier="Lusid", model_name="SimpleStatic", instrument_type="Bond"
                )
            ]
        ),
    )

    upsert_configuration_recipe_response = recipes_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
    
    return upsert_configuration_recipe_response


def create_txn_types(api_factory):
    system_configuration = api_factory.build(lusid.api.SystemConfigurationApi)

    response = system_configuration.set_transaction_configuration_source(
        source="abor",
        set_transaction_configuration_source_request=[
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FundsIn",
                        description="Deposit New Funds",
                        transaction_class="CashTransfers",
                        transaction_group="abor",
                        transaction_roles="Longer",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Subscription",
                        movement_types="CashReceivable",
                        side="Side1",
                        direction=1,
                        movement_options=[],
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="Subscription",
                        movement_types="Capital",
                        side="Side1",
                        direction=1,
                        movement_options=[],
                    )
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FundsOut",
                        description="Deposit New Funds",
                        transaction_class="CashTransfers",
                        transaction_group="abor",
                        transaction_roles="Shorter",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Redemption",
                        movement_types="CashReceivable",
                        side="Side1",
                        direction=-1,
                        movement_options=[],
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="Redemption",
                        movement_types="Capital",
                        side="Side1",
                        direction=-1,
                        movement_options=[],
                    )
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="Buy",
                        description="Buy",
                        transaction_class="Buy",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Bought",
                        movement_types="StockMovement",
                        side="Side1",
                        direction=1,
                        movement_options=[],
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="CashInvested",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=-1,
                        movement_options=[],
                    ),
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="Sell",
                        description="Sell",
                        transaction_class="Sell",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Sale",
                        movement_types="StockMovement",
                        side="Side1",
                        direction=-1,
                        movement_options=[],
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="CashProceeds",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=1,
                        movement_options=[],
                    ),
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FxSpotBuy",
                        description="FxSpotBuy",
                        transaction_class="FxSpotBuy",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="FxSpotBuyLeg",
                        movement_types="CashCommitment",
                        side="Side1",
                        direction=1,
                        movement_options=[],
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="FxSpotSellLeg",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=-1,
                        movement_options=[],
                    ),
                ],
            ),
        ],
    )
    
    return response

token = RefreshingToken()
secrets_path = os.getenv("FBN_SECRETS_PATH")

lusid_api_factory = lusid.utilities.ApiClientFactory(
    token=token,
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

atlas = lm.get_atlas(token=token)

create_recipe(lusid_api_factory)
create_txn_types(lusid_api_factory)

## A quick word on the luminesce magic command...

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` command is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.

Under the hood, when you run a cell with the `%%luminesce` command, Jupyter will pass the cell contents to this function:

```python

import os
from IPython.core.magic import (register_line_cell_magic)
from lumipy.client import Client

@register_line_cell_magic
def luminesce(line, cell=None):
    query = cell if cell is not None else line

    lm_client = Client(token=token, api_url=lumi_url)

    df = lm_client.query_and_fetch(query)
            
    return df
```

## Create a portfolio

First we create a portfolio to hold our Equity/Bond transactions and positions. 

In [2]:
%%luminesce

/*

------------------------------
Create a Transaction portfolio
------------------------------

In this snippet we load a Transaction portfolio into LUSID.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

-- Step 1: Define the portfolio details

@@scope = select 'luminesce-examples';
@@portfolioCode = select 'aborPortfolio';
@@writeAction = select 'Upsert';

@createPortfolio = select 'Transaction' as PortfolioType,
@@scope as PortfolioScope,
@@portfolioCode as PortfolioCode,
@@scope  as InstrumentScopes,
@@portfolioCode as DisplayName,
@@portfolioCode  as Description,
#2000-01-01# as Created,
'' as SubHoldingKeys,
'GBP' as BaseCurrency,
@@writeAction as WriteAction
;

-- Step 2: Load portfolio into LUSID

select *
from Lusid.Portfolio.Writer
where ToWrite = @createPortfolio;


,PortfolioScope,PortfolioCode,PortfolioType,BaseCurrency,DisplayName,Description,ParentPortfolioScope,ParentPortfolioCode,CorporateActionSourceScope,CorporateActionSourceId,...,SubHoldingKeys,Created,InstrumentScopes,AmortisationMethod,TransactionTypeScope,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,aborPortfolio,Transaction,GBP,aborPortfolio,aborPortfolio,NaN,NaN,NaN,NaN,...,NaN,2000-01-01,luminesce-examples,NoAmortisation,default,Update,2023-10-24 17:11:26.019,0,NaN,NaN


## Create equity instruments

Create some equity instruments using the LUSID `Equity` model.

In [3]:
%%luminesce

/*

------------------------------
Create some equity instruments
------------------------------

In this snippet we create some Equity instruments into LUSID.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

-- Step 1: Define the equity instruments

@@scope = select 'luminesce-examples';

@instrumentsData= 
values
('Tesco', 'FBNABOR001', 'GBP'),
('Sainsburys', 'FBNABOR002', 'GBP'),
('Walmart', 'FBNABOR003', 'USD'),
('Wholefoods', 'FBNABOR004', 'USD'),
('Waitrose', 'FBNABOR005', 'GBP');

@instrumentsForUpload = select
column1 as DisplayName,
column2 as ClientInternal,
column3 as DomCcy,
@@scope as Scope
from @instrumentsData;

-- Step 2: Upload the transformed data into LUSID

select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @instrumentsForUpload;


,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail,WriteAction,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,...,State,AsAtCreated,UserIdCreated,RequestIdCreated,AsAtModified,UserIdModified,RequestIdModified,AsAtVersionNumber,Identifiers,DomCcy
0,2023-09-15 10:02:52.945,0,NaN,NaN,Upsert,LUID_00003T0H,NaN,NaN,NaN,NaN,...,Active,2023-08-21 11:52:11.060,00u681arbmRvUDxQN2p7,3a529198-69de-49e1-9e7e-a50e92b53ba0,2023-10-19 10:00:54.932,00u681arbmRvUDxQN2p7,979bcfbb-9ae9-4e38-9d6d-7d749ea4c84a,23,NaN,USD
1,2023-08-21 11:52:11.060,0,NaN,NaN,Upsert,LUID_00003T0I,NaN,NaN,NaN,NaN,...,Active,2023-08-21 11:52:11.060,00u681arbmRvUDxQN2p7,3a529198-69de-49e1-9e7e-a50e92b53ba0,2023-10-19 10:00:54.932,00u681arbmRvUDxQN2p7,979bcfbb-9ae9-4e38-9d6d-7d749ea4c84a,20,NaN,GBP
2,2023-08-21 11:52:11.060,0,NaN,NaN,Upsert,LUID_00003T0F,NaN,NaN,NaN,NaN,...,Active,2023-08-21 11:52:11.060,00u681arbmRvUDxQN2p7,3a529198-69de-49e1-9e7e-a50e92b53ba0,2023-10-19 10:00:54.932,00u681arbmRvUDxQN2p7,979bcfbb-9ae9-4e38-9d6d-7d749ea4c84a,20,NaN,GBP
3,2023-08-21 11:52:11.060,0,NaN,NaN,Upsert,LUID_00003T0E,NaN,NaN,NaN,NaN,...,Active,2023-08-21 11:52:11.060,00u681arbmRvUDxQN2p7,3a529198-69de-49e1-9e7e-a50e92b53ba0,2023-10-19 10:00:54.932,00u681arbmRvUDxQN2p7,979bcfbb-9ae9-4e38-9d6d-7d749ea4c84a,20,NaN,GBP
4,2023-08-21 11:52:11.060,0,NaN,NaN,Upsert,LUID_00003T0G,NaN,NaN,NaN,NaN,...,Active,2023-08-21 11:52:11.060,00u681arbmRvUDxQN2p7,3a529198-69de-49e1-9e7e-a50e92b53ba0,2023-10-19 10:00:54.932,00u681arbmRvUDxQN2p7,979bcfbb-9ae9-4e38-9d6d-7d749ea4c84a,20,NaN,USD


## Create bond instruments

Create a `Bond` instrument using the Bond model. Here we create some simple UK and US government bonds.

In [4]:
%%luminesce

/*

-----------------------
Create Bond instruments
-----------------------

In this snippet we create Bond instruments.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

@@scope = select 'luminesce-examples';

-- Step 1: Define the bond instruments

@bondsData= 
values
('US BOND 4% 01/01/2033', 'FBNBND001',  #2023-01-01#,  0.04, 'USD', '1Y', 'ActAct', #2033-01-01#),
('US BOND 7% 01/01/2028', 'FBNBND002',  #2023-01-01#,  0.07, 'USD', '1Y', 'ActAct', #2028-01-01#),
('UK BOND 3% 01/01/2033', 'FBNBND003',  #2023-01-01#,  0.03, 'GBP', '1Y', 'ActAct', #2033-01-01#),
('UK BOND 8% 01/01/2028', 'FBNBND004',  #2023-01-01#,  0.08, 'GBP', '1Y', 'ActAct', #2028-01-01#);


@bondsDataForUpload = 
select
Column1 as DisplayName,
column2 as ClientInternal,
column3 as StartDate,
column4 as CouponRate,
column5 as DomCcy,
column5 as FlowConventionsCurrency,
column6 as FlowConventionsPaymentFrequency,
column7 as FlowConventionsDayCountConvention,
'MF' as FlowConventionsRollConvention,
column5 as FlowConventionsPaymentCalendars,
column5 as FlowConventionsResetCalendars,
0 as FlowConventionsSettleDays,
0 as FlowConventionsResetDays,
1 as Principal,
column8 as MaturityDate,
@@scope as Scope
from @bondsData;

-- Step 2: Upload the transformed data into LUSID

select *
from Lusid.Instrument.Bond.Writer
where ToWrite = @bondsDataForUpload;


,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail,WriteAction,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,...,FirstCouponPayDate,CalculationType,RoundingConventions1FaceValue,RoundingConventions1Precision,RoundingConventions1RoundingTarget,RoundingConventions1RoundingType,RoundingConventions2FaceValue,RoundingConventions2Precision,RoundingConventions2RoundingTarget,RoundingConventions2RoundingType
0,2023-08-21 11:52:14.004,0,NaN,NaN,Upsert,LUID_00003T0L,NaN,NaN,NaN,NaN,...,NaN,Standard,0,0,NaN,NaN,0,0,NaN,NaN
1,2023-09-15 10:02:54.759,0,NaN,NaN,Upsert,LUID_00003T0M,NaN,NaN,NaN,NaN,...,NaN,Standard,0,0,NaN,NaN,0,0,NaN,NaN
2,2023-08-21 11:52:14.004,0,NaN,NaN,Upsert,LUID_00003T0K,NaN,NaN,NaN,NaN,...,NaN,Standard,0,0,NaN,NaN,0,0,NaN,NaN
3,2023-08-21 11:52:14.004,0,NaN,NaN,Upsert,LUID_00003T0J,NaN,NaN,NaN,NaN,...,NaN,Standard,0,0,NaN,NaN,0,0,NaN,NaN


## Create instrument properties

Create some instrument properties which we'll assign to the instruments. We create properties for:

* Sector
* Asset Class 
* Internal rating

These properties are later used in Posting Rules for the ABOR.

In [5]:
%%luminesce

/*

----------------------------
Create instrument properties
----------------------------

In this snippet we create some instrument properties.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

@@scope = select 'luminesce-examples';

-- Step 1: Define the property definitions

@newProperties =
values
('Instrument', @@scope, 'Sector', 'string'),
('Instrument', @@scope, 'AssetClass', 'string'),
('Instrument', @@scope, 'InternalRating', 'number');

@propertyDefinitions =
select 
Column1 as [Domain], 
Column2 as [PropertyScope], 
Column3 as [PropertyCode], 
Column3 as [DisplayName], 
'Property' as [ConstraintStyle],
'system' as [DataTypeScope],
column4 as [DataTypeCode]
from @newProperties;

-- Step 2: Load property definitions

select *
from Lusid.Property.Definition.Writer
where ToWrite = @propertyDefinitions;


,Domain,PropertyScope,PropertyCode,DisplayName,Description,Lifetime,ConstraintStyle,Required,DerivationFormula,DataTypeScope,DataTypeCode,WriteAsAt,WriteAction,WriteErrorCode,WriteError,WriteErrorDetail
0,Instrument,luminesce-examples,AssetClass,AssetClass,NaN,Perpetual,Property,False,NaN,system,string,2023-10-24 17:11:26.019,Update,0,NaN,NaN
1,Instrument,luminesce-examples,InternalRating,InternalRating,NaN,Perpetual,Property,False,NaN,system,number,2023-10-24 17:11:26.019,Update,0,NaN,NaN
2,Instrument,luminesce-examples,Sector,Sector,NaN,Perpetual,Property,False,NaN,system,string,2023-10-24 17:11:26.019,Update,0,NaN,NaN


##  Assign properties to instrument

Then assign properties to each of the instruments.

In [6]:
%%luminesce

/*

--------------------------------
Assign properties to instruments
--------------------------------

In this snippet we assign properties to instruments.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup the property definitions referenced below 
    2. Setup the instruments referenced below

*/

@@scope = select 'luminesce-examples';

-- Step 1: Define the property definitions

@newProperties =
values

-- Sectors
('FBNABOR001', @@scope, 'Sector', 'Consumer'),
('FBNABOR002', @@scope, 'Sector', 'Consumer'),
('FBNABOR003', @@scope, 'Sector', 'Consumer'),
('FBNABOR004', @@scope, 'Sector', 'Consumer'),
('FBNABOR005', @@scope, 'Sector', 'Consumer'),
('FBNBND001', @@scope, 'Sector', 'Government'),
('FBNBND002', @@scope, 'Sector', 'Government'),
('FBNBND003', @@scope, 'Sector', 'Government'),
('FBNBND004', @@scope, 'Sector', 'Government'),

-- Country
('FBNABOR001', @@scope, 'AssetClass', 'Common Stock'),
('FBNABOR002', @@scope, 'AssetClass', 'Common Stock'),
('FBNABOR003', @@scope, 'AssetClass', 'Common Stock'),
('FBNABOR004', @@scope, 'AssetClass', 'Common Stock'),
('FBNABOR005', @@scope, 'AssetClass', 'Common Stock'),
('FBNBND001', @@scope, 'AssetClass', 'Government Bond'),
('FBNBND002', @@scope, 'AssetClass', 'Government Bond'),
('FBNBND003', @@scope, 'AssetClass', 'Government Bond'),
('FBNBND004', @@scope, 'AssetClass', 'Government Bond'),

-- Internal ratings
('FBNABOR001', @@scope, 'InternalRating', 8),
('FBNABOR002', @@scope, 'InternalRating', 8),
('FBNABOR003', @@scope, 'InternalRating', 9),
('FBNABOR004', @@scope, 'InternalRating', 7),
('FBNABOR005', @@scope, 'InternalRating', 7),
('FBNBND001', @@scope, 'InternalRating', 8),
('FBNBND002', @@scope, 'InternalRating', 8),
('FBNBND003', @@scope, 'InternalRating', 6),
('FBNBND004', @@scope, 'InternalRating', 7);

@instProperties =
select 
column1 as EntityId, 
'ClientInternal' as EntityIdType, 
'Instrument' as Domain,
Column2 as PropertyScope, 
Column3 as PropertyCode,
Column4 as Value,
@@scope as EntityScope
from @newProperties;

-- Upload the transformed data into LUSID

select *
from Lusid.Property.Writer
where ToWrite = @instProperties
limit 5;


,Domain,EntityIdType,EntityScope,EntityId,PropertyScope,PropertyCode,Value,Unit,EffectiveFrom,EffectiveUntil,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,Instrument,ClientInternal,luminesce-examples,FBNBND003,luminesce-examples,AssetClass,Government Bond,NaN,NaN,NaN,Upsert,2023-10-24 17:11:34.173,0,NaN,NaN
1,Instrument,ClientInternal,luminesce-examples,FBNBND003,luminesce-examples,InternalRating,6,NaN,NaN,NaN,Upsert,2023-10-24 17:11:34.173,0,NaN,NaN
2,Instrument,ClientInternal,luminesce-examples,FBNBND003,luminesce-examples,Sector,Government,NaN,NaN,NaN,Upsert,2023-10-24 17:11:34.173,0,NaN,NaN
3,Instrument,ClientInternal,luminesce-examples,FBNBND001,luminesce-examples,AssetClass,Government Bond,NaN,NaN,NaN,Upsert,2023-10-24 17:11:34.173,0,NaN,NaN
4,Instrument,ClientInternal,luminesce-examples,FBNBND001,luminesce-examples,InternalRating,8,NaN,NaN,NaN,Upsert,2023-10-24 17:11:34.173,0,NaN,NaN


## Create transactions

Create transactions on these instruments:

* We create Buys and Sells to simulate the realised P&L
* The transactions are in portfolio base and non-base currency to see impact of FX gain/loss


In [7]:
%%luminesce

/*

------------------
Create transactions
------------------

In this snippet we create some Transactions.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup a portfolio with scope/code per below
    2. Setup Buy and Sell Transaction Types
    3. Setup instruments with the ClientInternal instrument IDs referenced below

*/


@@scope = select 'luminesce-examples';
@@portfolioCode = select 'aborPortfolio';

-- Step 1: Define some transactions

@transactions = 
values

-- Equity Transactions
(@@scope, @@portfolioCode, 'txn_001', 'Buy', '2023-01-01', '2023-01-03', 1000, 10, 10000, 'GBP', 'FBNABOR001', 1),
(@@scope, @@portfolioCode, 'txn_002', 'Buy', '2023-01-01', '2023-01-03', 2000, 12, 20000, 'GBP', 'FBNABOR002', 1),
(@@scope, @@portfolioCode, 'txn_003', 'Buy', '2023-01-01', '2023-01-03', 3000, 13, 30000, 'USD', 'FBNABOR003', 0.8),
(@@scope, @@portfolioCode, 'txn_004', 'Buy', '2023-01-01', '2023-01-03', 4000, 14, 40000, 'USD', 'FBNABOR004', 0.8),
(@@scope, @@portfolioCode, 'txn_005', 'Buy', '2023-01-01', '2023-01-03', 5000, 15, 50000, 'GBP', 'FBNABOR005', 1),
(@@scope, @@portfolioCode, 'txn_006', 'Sell', '2023-02-01', '2023-02-03', 1000, 21, 21000, 'GBP', 'FBNABOR001', 1),
(@@scope, @@portfolioCode, 'txn_007', 'Sell', '2023-02-01', '2023-02-03', 1000, 22, 22000, 'GBP', 'FBNABOR002', 1),
(@@scope, @@portfolioCode, 'txn_008', 'Sell', '2023-02-01', '2023-02-03', 1000, 23, 23000, 'USD', 'FBNABOR003', 0.78),
(@@scope, @@portfolioCode, 'txn_009', 'Sell', '2023-02-01', '2023-02-03', 1000, 24, 24000, 'USD', 'FBNABOR004', 0.78),
(@@scope, @@portfolioCode, 'txn_010', 'Sell', '2023-02-01', '2023-02-03', 1000, 25, 25000, 'GBP', 'FBNABOR005', 1),

--Bond transactions
(@@scope, @@portfolioCode, 'txn_011', 'Buy', '2023-01-01', '2023-01-03', 100000, 100, 100000, 'USD', 'FBNBND001', 0.8),
(@@scope, @@portfolioCode, 'txn_012', 'Buy', '2023-01-01', '2023-01-03', 200000, 98, 200000, 'GBP', 'FBNBND003', 1),
(@@scope, @@portfolioCode, 'txn_013', 'Sell', '2023-02-01', '2023-02-03', 50000, 100, 50000, 'USD', 'FBNBND001', 0.78),
(@@scope, @@portfolioCode, 'txn_014', 'Sell', '2023-02-01', '2023-02-03', 100000, 102, 100000, 'GBP', 'FBNBND003', 1)
;


-- Step 2: Load transactions into LUSID

@createTransactions = 
select
column1 as PortfolioScope,
column2 as PortfolioCode,
column3 as TxnId,
column4 as Type,
column5 as TransactionDate,
column6 as SettlementDate,
column7 as Units,
column8 as TradePrice,
column9 as TotalConsideration,
column10 as SettlementCurrency,
column11 as ClientInternal,
column12 as TradeToPortfolioRate,
'abor' as Source
from @transactions;

-- Upload the transformed data into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @createTransactions;


,PortfolioScope,PortfolioCode,DisplayName,TxnId,Type,TransactionDate,SettlementDate,Units,TradePrice,TradePriceType,...,CompositeFigi,ShareClassFigi,Wertpapier,RIC,ABORTransactionID,WriteAsAt,WriteAction,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,aborPortfolio,aborPortfolio,txn_001,Buy,2023-01-01,2023-01-03,1000,10,Price,...,NaN,NaN,NaN,NaN,NaN,2023-10-24 17:11:37.187,Upsert,0,NaN,NaN
1,luminesce-examples,aborPortfolio,aborPortfolio,txn_002,Buy,2023-01-01,2023-01-03,2000,12,Price,...,NaN,NaN,NaN,NaN,NaN,2023-10-24 17:11:37.187,Upsert,0,NaN,NaN
2,luminesce-examples,aborPortfolio,aborPortfolio,txn_003,Buy,2023-01-01,2023-01-03,3000,13,Price,...,NaN,NaN,NaN,NaN,NaN,2023-10-24 17:11:37.187,Upsert,0,NaN,NaN
3,luminesce-examples,aborPortfolio,aborPortfolio,txn_004,Buy,2023-01-01,2023-01-03,4000,14,Price,...,NaN,NaN,NaN,NaN,NaN,2023-10-24 17:11:37.187,Upsert,0,NaN,NaN
4,luminesce-examples,aborPortfolio,aborPortfolio,txn_005,Buy,2023-01-01,2023-01-03,5000,15,Price,...,NaN,NaN,NaN,NaN,NaN,2023-10-24 17:11:37.187,Upsert,0,NaN,NaN
5,luminesce-examples,aborPortfolio,aborPortfolio,txn_006,Sell,2023-02-01,2023-02-03,1000,21,Price,...,NaN,NaN,NaN,NaN,NaN,2023-10-24 17:11:37.187,Upsert,0,NaN,NaN
6,luminesce-examples,aborPortfolio,aborPortfolio,txn_007,Sell,2023-02-01,2023-02-03,1000,22,Price,...,NaN,NaN,NaN,NaN,NaN,2023-10-24 17:11:37.187,Upsert,0,NaN,NaN
7,luminesce-examples,aborPortfolio,aborPortfolio,txn_008,Sell,2023-02-01,2023-02-03,1000,23,Price,...,NaN,NaN,NaN,NaN,NaN,2023-10-24 17:11:37.187,Upsert,0,NaN,NaN
8,luminesce-examples,aborPortfolio,aborPortfolio,txn_009,Sell,2023-02-01,2023-02-03,1000,24,Price,...,NaN,NaN,NaN,NaN,NaN,2023-10-24 17:11:37.187,Upsert,0,NaN,NaN
9,luminesce-examples,aborPortfolio,aborPortfolio,txn_010,Sell,2023-02-01,2023-02-03,1000,25,Price,...,NaN,NaN,NaN,NaN,NaN,2023-10-24 17:11:37.187,Upsert,0,NaN,NaN


## Upload fx and instrument prices

In the following cells, we load fx and instrument prices. For the purposes of this notebook we simulate some real looking prices. You can of course replace these prices with your own "real" prices from your data vendor. 

In this section, we do some preliminary data wrangling with python but the quote upload ultimately happens via Luminesce/Lumipy like the other cells in this notebook.

In [8]:
start_date = "2023-01-01"
end_date = "2023-04-01"
provider = "Lusid"
field = "mid"
scope = "luminesce-examples"

default_range = np.random.default_rng()

quote_provider_columns = [
    "QuoteEffectiveAt",
    "Unit",
    "Instrumentid",
    "InstrumentIdType",
    "QuoteType",
    "Provider",
    "Field",
    "QuoteScope",
    "ScaleFactor",
]

date_range = pd.date_range(start=start_date, end=end_date)

Simulate some GBP/USD and GBP/EUR rates for Q1 2023

In [9]:
for start_price, ccy_pair, scale_factor in [
    (1.2097, "GBP/USD", 1),
    (1.1300, "GBP/EUR", 1),
]:
    fx_quotes = [
        (
            date,
            ccy_pair[-3:],
            ccy_pair,
            "CurrencyPair",
            "Rate",
            provider,
            field,
            scope,
            scale_factor,
        )
        for date in date_range
    ]

    df = pd.DataFrame(fx_quotes, columns=quote_provider_columns)

    df["Value"] = default_range.normal(start_price, 0.02, len(date_range))

    atlas.lusid_instrument_quote_writer(to_write=lm.from_pandas(df)).select("*").go(
        quiet=True
    )

Simulate some Equity and Bond price for Q1 2023

In [10]:
for instrument_id, start_price, ccy, scale_factor, avg_change in [
    ("FBNABOR001", 31, "GBP", 1, 0.5),
    ("FBNABOR002", 32, "GBP", 1, 0.4),
    ("FBNABOR003", 33, "GBP", 1, 0.3),
    ("FBNABOR004", 34, "USD", 1, 0.6),
    ("FBNABOR005", 34, "GBP", 1, 0.5),
    ("FBNBND001", 105, "USD", 100, 0.18),
    ("FBNBND002", 105, "USD", 100, 0.14),
    ("FBNBND003", 107, "GBP", 100, 0.9),
    ("FBNBND004", 98, "GBP", 100, 0.12),
]:
    instrument_quotes = [
        (
            date,
            ccy,
            instrument_id,
            "ClientInternal",
            "Price",
            provider,
            field,
            scope,
            scale_factor,
        )
        for date in date_range
    ]

    df = pd.DataFrame(instrument_quotes, columns=quote_provider_columns)

    df["Value"] = default_range.normal(start_price, avg_change, len(date_range))

    atlas.lusid_instrument_quote_writer(to_write=lm.from_pandas(df)).select("*").go(
        quiet=True
    )

## Create cash transactions

Create some cash transactions:

* Subscriptions
* Redemptions
* FxSpots

In [11]:
%%luminesce

/*

------------------
Create transaction
------------------

In this snippet we create some cash Transactions.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup a portfolio with scope/code per below
    2. Setup FundsIn Transaction Types

*/


@@scope = select 'luminesce-examples';
@@portfolioCode = select 'aborPortfolio';



/*  
    Step 1: Define some cash transactions
    
    In the step below, we create a transaction type called AborFundsIn which is configured to create
    Capital movements in the Journal Entry

*/

@transactions = 
values
(@@scope, @@portfolioCode, 'csh_001', 'FundsIn', '2023-01-01', '2023-01-03', 1000000, 1, 1000000, 'GBP', 'CCY_GBP', 1, 1),
(@@scope, @@portfolioCode, 'csh_002', 'FundsIn', '2023-01-01', '2023-01-03', 1000000, 1, 1000000, 'USD', 'CCY_USD', 0.79, 1),
(@@scope, @@portfolioCode, 'csh_003', 'FundsOut', '2023-02-01', '2023-02-03', 200000, 1, 200000, 'GBP', 'CCY_GBP', 1, 1),
(@@scope, @@portfolioCode, 'csh_004', 'FundsOut', '2023-02-01', '2023-02-03', 200000, 1, 200000, 'USD', 'CCY_USD', 0.81, 1),

--Fx Spots
(@@scope, @@portfolioCode, 'txn_015', 'FxSpotBuy', '2023-02-15', '2023-02-17', 100000, 1, 80000, 'GBP', 'CCY_USD', 0.8, 0.8),
(@@scope, @@portfolioCode, 'txn_016', 'FxSpotBuy', '2023-02-16', '2023-02-18', 50000, 1, 40000, 'GBP', 'CCY_USD', 0.8, 0.8)
;


@createTransactions = 
select
column1 as PortfolioScope,
column2 as PortfolioCode,
column3 as TxnId,
column4 as Type,
column5 as TransactionDate,
column6 as SettlementDate,
column7 as Units,
column8 as TradePrice,
column9 as TotalConsideration,
column10 as SettlementCurrency,
column11 as LusidInstrumentId,
Column12 as TradeToPortfolioRate,
column13 as ExchangeRate,
'abor' as Source
from @transactions;

-- Step 2: Load transactions into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @createTransactions;


,PortfolioScope,PortfolioCode,DisplayName,TxnId,Type,TransactionDate,SettlementDate,Units,TradePrice,TradePriceType,...,CompositeFigi,ShareClassFigi,Wertpapier,RIC,ABORTransactionID,WriteAsAt,WriteAction,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,aborPortfolio,aborPortfolio,csh_001,FundsIn,2023-01-01,2023-01-03,1000000,1,Price,...,NaN,NaN,NaN,NaN,NaN,2023-08-21 11:52:08.417,Upsert,0,NaN,NaN
1,luminesce-examples,aborPortfolio,aborPortfolio,csh_002,FundsIn,2023-01-01,2023-01-03,1000000,1,Price,...,NaN,NaN,NaN,NaN,NaN,2023-08-21 11:52:08.417,Upsert,0,NaN,NaN
2,luminesce-examples,aborPortfolio,aborPortfolio,csh_003,FundsOut,2023-02-01,2023-02-03,200000,1,Price,...,NaN,NaN,NaN,NaN,NaN,2023-08-21 11:52:08.417,Upsert,0,NaN,NaN
3,luminesce-examples,aborPortfolio,aborPortfolio,csh_004,FundsOut,2023-02-01,2023-02-03,200000,1,Price,...,NaN,NaN,NaN,NaN,NaN,2023-08-21 11:52:08.417,Upsert,0,NaN,NaN
4,luminesce-examples,aborPortfolio,aborPortfolio,txn_015,FxSpotBuy,2023-02-15,2023-02-17,100000,1,Price,...,NaN,NaN,NaN,NaN,NaN,2023-08-21 11:52:08.417,Upsert,0,NaN,NaN
5,luminesce-examples,aborPortfolio,aborPortfolio,txn_016,FxSpotBuy,2023-02-16,2023-02-18,50000,1,Price,...,NaN,NaN,NaN,NaN,NaN,2023-08-21 11:52:08.417,Upsert,0,NaN,NaN


## Create a chart of accounts

Once the portfolio has been setup, we now move onto the ABOR setup. First we create a Chart of Accounts.

In [12]:
%%luminesce

/*

--------------------------
Create a Chart of Accounts
--------------------------

In this snippet we create a Chart of Accounts.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

-- Step 1: Define the Chart of Accounts

@@scope = select 'luminesce-examples';
@@code = select 'standardChartOfAccounts';
@@name = select 'Standard Chart Of Accounts';
@@writeAction = select 'Upsert';

@chartOfAccounts =
select
@@scope as ChartOfAccountsScope,
@@code as ChartOfAccountsCode,
@@name as DisplayName,
@@name as Description,
@@writeAction as WriteAction;

-- Step 2: Upload Chart of Account into LUSID

select * from Lusid.ChartOfAccounts.Writer where ToWrite = @chartOfAccounts;


,ChartOfAccountsScope,ChartOfAccountsCode,DisplayName,Description,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,standardChartOfAccounts,Standard Chart Of Accounts,Standard Chart Of Accounts,Update,2023-10-24 17:12:08.833,0,NaN,NaN


## Add accounts to chart of accounts

Next we add some accounts to the Chart of Account.

In [13]:
%%luminesce

/*

---------------------------------
Add Accounts to Chart of Accounts
---------------------------------

In this snippet we add Accounts to a Chart of Accounts.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup a Chart of Accounts with the scope/code referenced below

*/

-- Step 1: Define some accounts

@@scope = select 'luminesce-examples';
@@code = select 'standardChartOfAccounts';

@accounts = values
-- Investments
('A0001-Investments', 'Asset'),

-- Cash, commitments
('A0002-Settled-Cash', 'Asset'),

-- Sales and purchases for settlement
('A0003-Sales-To-Settle', 'Asset'),
('A0004-Purchases-To-Settle', 'Asset'),
('A0005-Long-FX-To-Settle', 'Asset'),
('A0006-Short-FX-To-Settle', 'Asset'),

--Capital
('A0007-Capital', 'Asset'),

-- Gains and Losses
('A0008-Realised-Market-Gains', 'Income'),
('A0009-Realised-Fx-Gains', 'Income'),
('A0010-UnrealisedGains', 'Income'),

-- Subs, reds and accruals
('A0011-Accruals', 'Income'),
('A0012-Subscriptions', 'Asset'),
('A0013-Redemptions', 'Asset'),

-- Unknown catch alls
('A0101-Unknown-NA',  'Asset'),
('A0102-Unknown-PL',  'Revenue'),
('A0103-Unknown-CA', 'Capital');

@chartsOfAccountsAccounts = select
@@scope as ChartOfAccountsScope,
@@code as ChartOfAccountsCode,
column1 as AccountCode,
column1 as Description,
column2 as Type,
'Manual' as Control,
'Active' as Status
from @accounts;

-- Step 2: Assign Accounts onto a ChartOfAccount

select * from Lusid.ChartOfAccounts.Account.Writer 
where ToWrite = @chartsOfAccountsAccounts;


,ChartOfAccountsScope,ChartOfAccountsCode,AccountCode,Description,Type,Control,Status,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,standardChartOfAccounts,A0001-Investments,A0001-Investments,Asset,Manual,Active,Upsert,NaN,0,NaN,NaN
1,luminesce-examples,standardChartOfAccounts,A0002-Settled-Cash,A0002-Settled-Cash,Asset,Manual,Active,Upsert,NaN,0,NaN,NaN
2,luminesce-examples,standardChartOfAccounts,A0003-Sales-To-Settle,A0003-Sales-To-Settle,Asset,Manual,Active,Upsert,NaN,0,NaN,NaN
3,luminesce-examples,standardChartOfAccounts,A0004-Purchases-To-Settle,A0004-Purchases-To-Settle,Asset,Manual,Active,Upsert,NaN,0,NaN,NaN
4,luminesce-examples,standardChartOfAccounts,A0005-Long-FX-To-Settle,A0005-Long-FX-To-Settle,Asset,Manual,Active,Upsert,NaN,0,NaN,NaN
5,luminesce-examples,standardChartOfAccounts,A0006-Short-FX-To-Settle,A0006-Short-FX-To-Settle,Asset,Manual,Active,Upsert,NaN,0,NaN,NaN
6,luminesce-examples,standardChartOfAccounts,A0007-Capital,A0007-Capital,Asset,Manual,Active,Upsert,NaN,0,NaN,NaN
7,luminesce-examples,standardChartOfAccounts,A0008-Realised-Market-Gains,A0008-Realised-Market-Gains,Income,Manual,Active,Upsert,NaN,0,NaN,NaN
8,luminesce-examples,standardChartOfAccounts,A0009-Realised-Fx-Gains,A0009-Realised-Fx-Gains,Income,Manual,Active,Upsert,NaN,0,NaN,NaN
9,luminesce-examples,standardChartOfAccounts,A0010-UnrealisedGains,A0010-UnrealisedGains,Income,Manual,Active,Upsert,NaN,0,NaN,NaN


## Create posting module

Then we assign some posting rules to the Chart of Account which control how each JE line gets assigned to each account.

In [14]:
%%luminesce

/*

----------------------
Create posting modules
----------------------

In this snippet we create a posting module.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup a Chart of Accounts with the scope/code referenced below

*/

-- Step 1: Define a posting module

@@scope = select 'luminesce-examples';
@@coaCode = select 'standardChartOfAccounts';
@@postingModuleCode = select 'standardPostingModule';
@@writeAction = select 'Upsert';

@postingModule = 
select
@@postingModuleCode as PostingModuleCode,
@@coaCode as ChartOfAccountsCode,
@@scope as ChartOfAccountsScope, 
'Active' as Status,
'Daily NAV' as DisplayName,
'Posting module for daily NAV' as Description,
@@writeAction as WriteAction;

-- Step 2: Upload posting module into LUSID

select * from Lusid.PostingModule.Writer where ToWrite = @postingModule;


,ChartOfAccountsScope,ChartOfAccountsCode,PostingModuleCode,Description,DisplayName,Status,WriteAsAt,WriteAction,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,standardChartOfAccounts,standardPostingModule,Posting module for daily NAV,Daily NAV,Active,2023-10-24 17:12:12.044,Upsert,0,NaN,NaN


## Create posting rules

In [15]:
%%luminesce

/*

--------------------
Create posting rules
--------------------

In this snippet we create some posting rules.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup a Posting Module wuth the scope/code referenced below


*/

-- Step 1: Define posting rules

@@scope = select 'luminesce-examples';
@@coaCode = select 'standardChartOfAccounts';
@@postingModuleCode = select 'standardPostingModule';

@rules_filters = values
(
    'Rule-001', 
    'EconomicBucket startswith ''NA'' 
        and HoldType eq ''P''',
    'A0001-Investments',
    1
),
(
    'Rule-002',
    'EconomicBucket startswith ''NA'' and HoldType eq ''B''',
    'A0002-Settled-Cash',
    2
),
(
    'Rule-003', 
    'EconomicBucket startswith ''NA_'' 
        and MovementName eq ''CashProceeds'' 
        and HoldType neq ''B''',
    'A0003-Sales-To-Settle',
    3
),
(
    'Rule-004', 
    'EconomicBucket startswith ''NA_'' 
        and MovementName eq ''CashInvested'' 
        and HoldType neq ''B''',
    'A0004-Purchases-To-Settle',
    4
),
(
    'Rule-005', 
    'EconomicBucket startswith ''NA_'' 
        and MovementName eq ''FxSpotBuyLeg'' 
        and HoldType neq ''B''',
    'A0005-Long-FX-To-Settle',
    5
),
(
    'Rule-006', 
    'EconomicBucket startswith ''NA_'' 
        and MovementName eq ''FxSpotSellLeg'' 
        and HoldType neq ''B''',
    'A0006-Short-FX-To-Settle',
    6
),
(
    'Rule-007',
    'EconomicBucket eq ''CA_Capital''', 
    'A0007-Capital',
    7
),
(
    'Rule-008', 
    'EconomicBucket eq ''PL_RealPriceGL'' and HoldType eq ''P''', 
    'A0008-Realised-Market-Gains',
    8
),
(
    'Rule-009', 
    'EconomicBucket eq ''PL_RealFXGL'' and HoldType in ''P'', ''B''', 
    'A0009-Realised-Fx-Gains',
    9
),
(
    'Rule-010', 
    'EconomicBucket startswith ''PL_Unreal''',
    'A0010-UnrealisedGains',
    10
),
(
    'Rule-011', 
    'EconomicBucket startswith ''PL_Accrued''',
    'A0011-Accruals',
    11
),
(
    'Rule-012', 
    'MovementName eq ''Subscription'' and HoldType neq ''B''',
    'A0012-Subscriptions',
    12
),
(
    'Rule-013', 
    'MovementName eq ''Redemption'' and HoldType neq ''B''',
    'A0013-Redemptions',
    13
),
(
    'Rule-101', 
    'EconomicBucket startswith ''NA''',
    'A0101-Unknown-NA',
    14
),
(
    'Rule-102', 
    'EconomicBucket startswith ''PL''',
    'A0102-Unknown-PL',
    15
),
(
    'Rule-103', 
    'EconomicBucket startswith ''CA''',
    'A0103-Unknown-CA',
    16
)
;


-- Step 2: Add posting rules to posting module

@postingRules = select 
@@scope as ChartOfAccountsScope,
@@coaCode as ChartOfAccountsCode,
@@postingModuleCode as PostingModuleCode,
column1 as RuleId,
column2 as RuleFilter,
column3 as GeneralLedgerAccountCode,
column4 as RulePriority
from @rules_filters;

select * from Lusid.PostingModule.Rule.Writer where ToWrite = @postingRules;


,ChartOfAccountsScope,ChartOfAccountsCode,PostingModuleCode,RulePriority,RuleId,AccountCode,RuleFilter,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,standardChartOfAccounts,standardPostingModule,1,Rule-001,A0001-Investments,EconomicBucket startswith 'NA' \n and H...,2023-10-24 17:12:13.180,0,NaN,NaN
1,luminesce-examples,standardChartOfAccounts,standardPostingModule,2,Rule-002,A0002-Settled-Cash,EconomicBucket startswith 'NA' and HoldType eq...,2023-10-24 17:12:13.180,0,NaN,NaN
2,luminesce-examples,standardChartOfAccounts,standardPostingModule,3,Rule-003,A0003-Sales-To-Settle,EconomicBucket startswith 'NA_' \n and ...,2023-10-24 17:12:13.180,0,NaN,NaN
3,luminesce-examples,standardChartOfAccounts,standardPostingModule,4,Rule-004,A0004-Purchases-To-Settle,EconomicBucket startswith 'NA_' \n and ...,2023-10-24 17:12:13.180,0,NaN,NaN
4,luminesce-examples,standardChartOfAccounts,standardPostingModule,5,Rule-005,A0005-Long-FX-To-Settle,EconomicBucket startswith 'NA_' \n and ...,2023-10-24 17:12:13.180,0,NaN,NaN
5,luminesce-examples,standardChartOfAccounts,standardPostingModule,6,Rule-006,A0006-Short-FX-To-Settle,EconomicBucket startswith 'NA_' \n and ...,2023-10-24 17:12:13.180,0,NaN,NaN
6,luminesce-examples,standardChartOfAccounts,standardPostingModule,7,Rule-007,A0007-Capital,EconomicBucket eq 'CA_Capital',2023-10-24 17:12:13.180,0,NaN,NaN
7,luminesce-examples,standardChartOfAccounts,standardPostingModule,8,Rule-008,A0008-Realised-Market-Gains,EconomicBucket eq 'PL_RealPriceGL' and HoldTyp...,2023-10-24 17:12:13.180,0,NaN,NaN
8,luminesce-examples,standardChartOfAccounts,standardPostingModule,9,Rule-009,A0009-Realised-Fx-Gains,EconomicBucket eq 'PL_RealFXGL' and HoldType i...,2023-10-24 17:12:13.180,0,NaN,NaN
9,luminesce-examples,standardChartOfAccounts,standardPostingModule,10,Rule-010,A0010-UnrealisedGains,EconomicBucket startswith 'PL_Unreal',2023-10-24 17:12:13.180,0,NaN,NaN


## Create an abor configuration

Next we create an Abor Configuration which binds a Chart of Accounts (with Posting Rules) to a given recipe.

In [16]:
%%luminesce

/*

----------------------------
Create an ABOR configuration
----------------------------

In this snippet we create an ABOR Configurtion.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup Chart of Accounts referenced below
    2. Setup a Posting Module wuth the scope/code referenced below
    3. Setup the recipe referenced below


*/

@@scope = select 'luminesce-examples';
@@chartOfAccountsCode = select 'standardChartOfAccounts';
@@code = select 'standardAborConfiguration';
@@writeAction = select 'Insert';
@@PostingModuleCodes = select 'standardPostingModule';

-- Step 1: Create an ABOR configuration

@aborConfigurationForUpload = select
@@code as AborConfigurationCode,
@@scope as AborConfigurationScope,
@@code as Description,
@@chartOfAccountsCode as ChartOfAccountsCode,
@@scope as ChartOfAccountsScope,
@@PostingModuleCodes as PostingModuleCodes,
@@code as DisplayName,
@@scope as RecipeScope,
'marketValue' as RecipeCode,
@@writeAction as WriteAction;

-- Step 2: Load ABOR configurtion into LUSID

select * from Lusid.AborConfiguration.Writer
where ToWrite = @aborConfigurationForUpload;



,AborConfigurationScope,AborConfigurationCode,RecipeScope,RecipeCode,DisplayName,ChartOfAccountsScope,ChartOfAccountsCode,Description,PostingModuleCodes,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,standardAborConfiguration,luminesce-examples,marketValue,standardAborConfiguration,luminesce-examples,standardChartOfAccounts,standardAborConfiguration,standardPostingModule,Insert,NaN,173,Could not create an AborConfiguration with id ...,Error creating AborConfiguration with id 'stan...


## Create abor

We also create an ABOR which binds an AborConfiguration to a given portfolio.

In [17]:
%%luminesce

/*

--------------
Create an ABOR
--------------

In this snippet we create an ABOR.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup Abor Configuration referenced below
    2. Setup portfolio referenced below


*/

@@scope = select 'luminesce-examples';
@@code = select 'standardAbor';
@@aborConfigCode = select 'standardAborConfiguration';
@@portfolioCode = select 'aborPortfolio';
@@writeAction = select 'Upsert';

-- Step 1: Define an ABOR

@aborForUpload = select
@@scope as AborScope,
@@code as AborCode,
@@code as DisplayName,
'SinglePortfolio' as PortfolioEntityType,
@@scope  as PortfolioScope,
@@portfolioCode as PortfolioCode,
@@portfolioCode as Description,
@@scope  as AborConfigurationScope,
@@aborConfigCode as AborConfigurationCode,
@@writeAction as WriteAction;

-- Step 2: Load ABOR into LUSID

select * from Lusid.Abor.Writer 
where ToWrite = @aborForUpload


,AborScope,AborCode,PortfolioEntityType,PortfolioScope,PortfolioCode,DisplayName,Description,AborConfigurationScope,AborConfigurationCode,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,standardAbor,SinglePortfolio,luminesce-examples,aborPortfolio,standardAbor,aborPortfolio,luminesce-examples,standardAborConfiguration,Update,2023-10-24 17:12:14.728,0,NaN,NaN


## Create journal entry lines

We can then run some JE (Journal Entry) lines against the ABOR.

In [18]:
%%luminesce

/*

-------------------------------
Create Journal Entry (JE) Lines
-------------------------------

In this snippet we create an ABOR.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Create the ABOR referenced below


*/

@@scope = select 'luminesce-examples';
@@code = select 'standardAbor';

--Step 1: Generate JE Lines

select * from Lusid.Abor.JELine
where AborScope = @@scope 
and AborCode = @@code
and StartDate = '2023-01-02'
and EndDate = '2023-03-03';


,AborScope,AborCode,PortfolioScope,PortfolioCode,AccountingDate,ActivityDate,GlCode,SourceType,SourceId,InstrumentId,...,BaseAmount,BaseCurrency,PostingModuleCode,PostingRule,AsAtDate,ActivitiesDescription,MovementName,HoldingType,EconomicBucket,Error
0,luminesce-examples,standardAbor,luminesce-examples,aborPortfolio,2023-01-03,2023-01-03,A0004-Purchases-To-Settle,LusidTransaction,txn_001,CCY_GBP,...,10000.00,GBP,standardPostingModule,Rule-004,2023-10-24,A0004-Purchases-To-Settle,CashInvested,C,NA_Cost,NaN
1,luminesce-examples,standardAbor,luminesce-examples,aborPortfolio,2023-01-03,2023-01-03,A0002-Settled-Cash,LusidTransaction,txn_001,CCY_GBP,...,-10000.00,GBP,standardPostingModule,Rule-002,2023-10-24,A0002-Settled-Cash,CashInvested,B,NA_Cost,NaN
2,luminesce-examples,standardAbor,luminesce-examples,aborPortfolio,2023-01-03,2023-01-03,A0004-Purchases-To-Settle,LusidTransaction,txn_002,CCY_GBP,...,20000.00,GBP,standardPostingModule,Rule-004,2023-10-24,A0004-Purchases-To-Settle,CashInvested,C,NA_Cost,NaN
3,luminesce-examples,standardAbor,luminesce-examples,aborPortfolio,2023-01-03,2023-01-03,A0002-Settled-Cash,LusidTransaction,txn_002,CCY_GBP,...,-20000.00,GBP,standardPostingModule,Rule-002,2023-10-24,A0002-Settled-Cash,CashInvested,B,NA_Cost,NaN
4,luminesce-examples,standardAbor,luminesce-examples,aborPortfolio,2023-01-03,2023-01-03,A0004-Purchases-To-Settle,LusidTransaction,txn_003,CCY_USD,...,24000.00,GBP,standardPostingModule,Rule-004,2023-10-24,A0004-Purchases-To-Settle,CashInvested,C,NA_Cost,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,luminesce-examples,standardAbor,luminesce-examples,aborPortfolio,2023-01-02,2023-01-02,A0010-UnrealisedGains,LusidValuation,2023-01-02T00:00:00.0000000+00:00,CCY_USD,...,-2502.88,GBP,standardPostingModule,Rule-010,2023-10-24,A0010-UnrealisedGains,CashInvested,C,PL_UnrealFXGL,NaN
128,luminesce-examples,standardAbor,luminesce-examples,aborPortfolio,2023-01-02,2023-01-02,A0012-Subscriptions,LusidValuation,2023-01-02T00:00:00.0000000+00:00,CCY_USD,...,-35028.77,GBP,standardPostingModule,Rule-012,2023-10-24,A0012-Subscriptions,Subscription,R,NA_UnrealFXGL,NaN
129,luminesce-examples,standardAbor,luminesce-examples,aborPortfolio,2023-01-02,2023-01-02,A0010-UnrealisedGains,LusidValuation,2023-01-02T00:00:00.0000000+00:00,CCY_USD,...,35028.77,GBP,standardPostingModule,Rule-010,2023-10-24,A0010-UnrealisedGains,Subscription,R,PL_UnrealFXGL,NaN
130,luminesce-examples,standardAbor,luminesce-examples,aborPortfolio,2023-03-03,2023-03-03,A0002-Settled-Cash,LusidValuation,2023-03-03T00:00:00.0000000+00:00,CCY_USD,...,33260.21,GBP,standardPostingModule,Rule-002,2023-10-24,A0002-Settled-Cash,MarkToMarket,B,NA_UnrealFXGL,NaN


## General ledger profile

Create a General Ledger profile.

In [19]:
%%luminesce

@@scope = select 'luminesce-examples';
@@chartOfAccountsCode = select 'standardChartOfAccounts';
@@generalLedgerProfileCode = select 'standardGeneralLedgerProfile';

@glProfile = select
@@scope as ChartOfAccountsScope,
@@chartOfAccountsCode as ChartOfAccountsCode,
@@chartOfAccountsCode as DisplayName,
@@generalLedgerProfileCode as GeneralLedgerProfileCode,
'Insert' as WriteAction
;

select * from Lusid.GeneralLedgerProfile.Writer
where ToWrite = @glProfile;

,ChartOfAccountsScope,ChartOfAccountsCode,GeneralLedgerProfileCode,Description,DisplayName,WriteAsAt,WriteAction,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,standardChartOfAccounts,standardGeneralLedgerProfile,NaN,standardChartOfAccounts,NaN,Insert,173,Could not create a General Ledger Profile with...,Error creating General Ledger Profile with cod...


## General Ledger Profile mapppings

Add some mappings to the General Ledger Profile.

In [20]:
%%luminesce

@@scope = select 'luminesce-examples';
@@chartOfAccountsCode = select 'standardChartOfAccounts';
@@generalLedgerProfileCode = select 'standardGeneralLedgerProfile';


@mappings = values
(
    'GeneralLedgerAccountCode eq ''A0001-Investments''', 
    'DefaultCurrency',
    'Properties[Instrument/luminesce-examples/AssetClass]',
    '',
    1
),
(
    'GeneralLedgerAccountCode eq ''A0002-Settled-Cash''', 
    'DefaultCurrency',
    Null,
    '',
    2
),
(
    'GeneralLedgerAccountCode eq ''A0003-Sales-To-Settle''', 
    'DefaultCurrency',
    Null,
    '',
    3
),
(
    'GeneralLedgerAccountCode eq ''A0004-Purchases-To-Settle''', 
    'DefaultCurrency',
    Null,
    '',
    4
),
(
    'GeneralLedgerAccountCode eq ''A0010-UnrealisedGains''', 
    'DefaultCurrency',
    Null,
    '',
    5
)
;

@mappingsToWrite = select  
@@chartOfAccountsCode as ChartOfAccountsCode,
@@scope as ChartOfAccountsScope,
@@generalLedgerProfileCode as GeneralLedgerProfileCode,
column1 as MappingFilter,
column2 as Level1,
column3 as Level2,
column5 as MappingPriority
from @mappings;

select * from Lusid.GeneralLedgerProfile.Mapping.Writer
where ToWrite = @mappingsToWrite;

,ChartOfAccountsScope,ChartOfAccountsCode,GeneralLedgerProfileCode,MappingPriority,MappingFilter,Level1,Level2,Level3,Level4,Level5,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,luminesce-examples,standardChartOfAccounts,standardGeneralLedgerProfile,1,GeneralLedgerAccountCode eq 'A0001-Investments',DefaultCurrency,Properties[Instrument/luminesce-examples/Asset...,NaN,NaN,NaN,2023-10-24 17:12:25.394,0,NaN,NaN
1,luminesce-examples,standardChartOfAccounts,standardGeneralLedgerProfile,2,GeneralLedgerAccountCode eq 'A0002-Settled-Cash',DefaultCurrency,NaN,NaN,NaN,NaN,2023-10-24 17:12:25.394,0,NaN,NaN
2,luminesce-examples,standardChartOfAccounts,standardGeneralLedgerProfile,3,GeneralLedgerAccountCode eq 'A0003-Sales-To-Se...,DefaultCurrency,NaN,NaN,NaN,NaN,2023-10-24 17:12:25.394,0,NaN,NaN
3,luminesce-examples,standardChartOfAccounts,standardGeneralLedgerProfile,4,GeneralLedgerAccountCode eq 'A0004-Purchases-T...,DefaultCurrency,NaN,NaN,NaN,NaN,2023-10-24 17:12:25.394,0,NaN,NaN
4,luminesce-examples,standardChartOfAccounts,standardGeneralLedgerProfile,5,GeneralLedgerAccountCode eq 'A0010-UnrealisedG...,DefaultCurrency,NaN,NaN,NaN,NaN,2023-10-24 17:12:25.394,0,NaN,NaN


## Generate a Trial Balance

Now we can generate a Trial Balance

In [21]:
%%luminesce

select 
GeneralLedgerProfileAccountCode,
Level1,
Level2,
AccountType,
Opening,
Closing,
Debit,
Credit
from Lusid.Abor.TrialBalance
where StartDate = '2023-01-02'
and EndDate = '2023-03-03'
and AborScope = 'luminesce-examples'
and GeneralLedgerProfileCode = 'standardGeneralLedgerProfile'
order by GeneralLedgerProfileAccountCode

,GeneralLedgerProfileAccountCode,Level1,Level2,AccountType,Opening,Closing,Debit,Credit
0,A0001-Investments,GBP,Common Stock,Asset,262844.40,163367.65,113367.65,-212844.40
1,A0001-Investments,USD,Common Stock,Asset,211204.30,149776.71,109776.71,-171204.30
2,A0001-Investments,USD,Government Bond,Asset,86422.00,43878.90,3878.90,-46422.00
3,A0001-Investments,GBP,Government Bond,Asset,213427.56,108311.30,8311.30,-113427.56
4,A0002-Settled-Cash,GBP,NaN,Asset,0.00,568000.00,1168000.00,-600000.00
5,A0002-Settled-Cash,USD,NaN,Asset,0.00,726829.49,1020620.21,-293790.72
6,A0003-Sales-To-Settle,GBP,NaN,Asset,0.00,0.00,168000.00,-168000.00
7,A0003-Sales-To-Settle,USD,NaN,Asset,0.00,0.00,75660.00,-75660.00
8,A0004-Purchases-To-Settle,GBP,NaN,Asset,-280000.00,0.00,280000.00,0.00
9,A0004-Purchases-To-Settle,USD,NaN,Asset,-140254.89,0.00,140254.89,0.00


## Run a check 

In the query below, we verify that the sum of Credits is equal the sum of the Debits.

In [22]:
%%luminesce

@tb = select sum(Debit) as SumDebits, sum(Credit) as SumCredits from Lusid.Abor.TrialBalance
where GeneralLedgerProfileCode = 'standardGeneralLedgerProfile'
and StartDate = '2023-01-02'
and EndDate = '2023-03-03'
and AborScope = 'luminesce-examples';

select SumDebits, SumCredits, round((SumDebits + SumCredits), 2) as [Check] from @tb;

,SumDebits,SumCredits,Check
0,4446196.69,-4446196.69,0
